In [1]:
import os

while not os.getcwd().endswith("hackathon"):
    os.chdir("..")
    print(f"Now in {os.getcwd()}")

Now in c:\Users\GiovanniGiacometti\Desktop\hackathon


In [2]:
from hackathon.utils.settings.settings_provider import SettingsProvider
import warnings
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

with warnings.catch_warnings(action="ignore"):
    from langchain_ibm import ChatWatsonx
import dotenv
from langchain.schema import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


dotenv.load_dotenv();

In [3]:
settings_provider = SettingsProvider()

In [4]:
llm = ChatWatsonx(
    model_id=settings_provider.get_ibm_model_name(),  # type: ignore
    url=settings_provider.get_ibm_endpoint_url(),  # type: ignore
    project_id=settings_provider.get_ibm_project_id(),  # type: ignore
)

In [5]:
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

embeddings = HuggingFaceEmbeddings(
    model_name=settings_provider.get_embeddings_model_name(),
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [6]:
# import faiss
# from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain_community.vectorstores import FAISS

# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

# vectorstore = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

In [7]:
vector_store = FAISS.load_local(
    "data/debug_vectorstore", embeddings, allow_dangerous_deserialization=True
)

In [8]:
# # from uuid import uuid4

# from langchain_core.documents import Document

# document_1 = Document(
#     page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
#     metadata={"source": "tweet", "options": ["pancakes", "eggs"]},
# )

# document_2 = Document(
#     page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
#     metadata={"source": "news", "options": ["weather", "forecast"]},
# )

# document_3 = Document(
#     page_content="Building an exciting new project with LangChain - come check it out!",
#     metadata={"source": "tweet", "options": ["LangChain", "project"]},
# )

# documents = [
#     document_1,
#     document_2,
#     document_3,
# ]
# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)
# vector_store.save_local(folder_path="data/debug_vectorstore")

In [9]:
# Alternative: If you want exact array match

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5, "fetch_k": vector_store.index.ntotal})

retriever.invoke(
    "Radici di Singolaavfavsavfdfdafrità",
    filter=lambda x: "Alessandra Quanti" in x.get("chef_name", ""),
)

[Document(id='d3a548ca-8f7d-44b3-9415-5801267b4c29', metadata={'header_1': 'Menu', 'header_2': 'Sinfonia Quantistica delle Stelle', 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'planet_name': 'Tatooine', 'licences': ['Psionica II', 'Temporale I', 'Gravitazionale I', 'Quantistica 13', 'Luce II']}, page_content='### Ingredienti  \nShard di Prisma Stellare\nLattuga Namecciana\nRadici di Singolarità  \n-----  \nFibra di Sintetex\nCarne di Balena spaziale\nTeste di Idra\nNettare di Sirena\nSale Temporale  \n### Tecniche  \nMarinatura Temporale Sincronizzata\nCottura Sottovuoto Frugale Energeticamente Negativa\nCottura a Vapore Termocinetica Multipla\nCottura a Vapore Ecodinamico Bilanciato'),
 Document(id='2ba8bbbf-ae68-41f2-ac1f-a4dd518019ad', metadata={'header_1': 'Menu', 'header_2': 'Evanescenza Quantica', 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'planet_name': 'Tat

In [10]:
from hackathon.models import MenuMetadata


def query_with_metadata(query: str, metadata: MenuMetadata):

    conditions = []

    if metadata.chef_name is not None:
        
        conditions.append(lambda x: metadata.chef_name in x.get("chef_name", ""))


    if metadata.restaurant_name is not None:
        conditions.append(lambda x: metadata.restaurant_name in x.get("restaurant_name", ""))

    if metadata.planet_name is not None:
        conditions.append(lambda x: metadata.planet_name in x.get("planet_name", ""))

    if len(metadata.licences):

        conditions.append(lambda x: any(lic in x.get("licences", []) for lic in metadata.licences))

    if len(conditions) == 0:
        filter_fn = None

    else:
        filter_fn = lambda x: any(cond(x) for cond in conditions) #noqa
    
    return retriever.invoke(
        query,
        filter=filter_fn
    )

In [11]:
query_ex = "Quali piatti del ristorante Cosmica Essenza sono preparati utilizzando i Funghi dell’Etere o la tecnica di Marinatura Temporale Sincronizzata?"

In [12]:
def clean_string(string: str):
    return string.lower().replace("_", " ").replace("'", " ")


def search_string_in_dict(string: str, key: str, dictionary: dict):
    search_string = clean_string(string)

    if key not in dictionary:
        return False

    if dictionary[key] is None:
        return True

    if isinstance(dictionary[key], str):
        return search_string in clean_string(dictionary[key])

    elif isinstance(dictionary[key], list):
        return search_string in [clean_string(x) for x in dictionary[key]]

    else:
        raise ValueError(f"Unsupported type {type(dictionary[key])}")


def filter_with_metadata_local(metadata: MenuMetadata):
    conditions = []
    fields = []

    if metadata.chef_name is not None:
        fields.append("chef_name")
        conditions.append(
            lambda doc_metadata: search_string_in_dict(
                string=metadata.chef_name, key="chef_name", dictionary=doc_metadata
            )
        )

    if metadata.restaurant_name is not None:
        fields.append("restaurant_name")
        conditions.append(
            lambda doc_metadata: search_string_in_dict(
                string=metadata.restaurant_name,
                key="restaurant_name",
                dictionary=doc_metadata,
            )
        )

    if metadata.planet_name is not None:
        fields.append("planet_name")
        conditions.append(
            lambda doc_metadata: search_string_in_dict(
                string=metadata.planet_name, key="planet_name", dictionary=doc_metadata
            )
        )

    if len(metadata.licences):
        fields.append("licences")
        conditions.append(
            lambda doc_metadata: any(
                search_string_in_dict(
                    string=lic, key="licences", dictionary=doc_metadata
                )
                for lic in metadata.licences
            )
        )

    if len(conditions) == 0:
        print("No metadata fields to filter")
        filter_fn = None

    else:
        print(f"Filtering documents with metadata fields: {fields}")
        filter_fn = lambda x: any(cond(x) for cond in conditions)  # noqa

    return filter_fn

In [13]:
from hackathon.session import SessionManager

retriever = SessionManager().vectorstore_manager.retriever

INFO:hackathon.managers.vectore_store:Initializing vectorstore...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:hackathon.managers.vectore_store:Vectorstore initialized successfully.


In [17]:
from hackathon.graph.nodes.retrieve import filter_with_metadata

docs = retriever.invoke(query_ex, filter=filter_with_metadata(
    MenuMetadata(
        chef_name="Alessandra",
)))


for doc in docs:

    print(doc.page_content)
    print("---_")

INFO:hackathon.graph.nodes.retrieve:Filtering documents with metadata fields: ['chef_name']


Preparato con l'abilità senza pari di Chef Alessandra Tempesti, il Galassia a Tavola è un viaggio sensoriale
attraverso lo spazio e il tempo, incarnato in un piatto che fonde estetica e sapore. Questa creazione
culinaria, concepita al "L'Architetto dell'Universo", è una vera e propria danza tra le stelle, un omaggio agli
ingredienti più rari e alle tecniche più sofisticate dell'arte gastronomica cosmica.  
Alla base del piatto si trovano strati delicati di Carne di Xenodonte, grigliati alla perfezione mediante la
tecnica di Grigliatura Elettro-Molecolare a Spaziatura Variabile, rivelando un sapore umami profondo e
stratificato. Su questa base terrosa si adagiano le Teste di Idra, cucinate con maestria utilizzando la Cottura
al Forno con Paradosso Temporale Cronospeculare per assicurare una succulenza che sfida le ere, mentre
la preparazione scrupolosa ne garantisce la totale sicurezza.  
Le consistenze aromatiche dei Funghi Orbitali, coltivati su satelliti artificiali, avvolgono la com

In [46]:
query_ex = "Quali piatti del ristorante Cosmica Essenza sono preparati utilizzando i Funghi dell’Etere o la tecnica di Marinatura Temporale Sincronizzata?"

In [243]:
import numpy as np


embedding_dim = vector_store.index.d
dummy_vector = np.zeros(embedding_dim)

# Retrieve all documents by setting k to the total number of documents
docs_and_scores = vector_store.similarity_search_with_score_by_vector(
    dummy_vector, k=vector_store.index.ntotal
)

list(filter(lambda x: "cosmica" in x if x else False, list(map(lambda x: x[0].metadata.get("restaurant_name"), docs_and_scores))))

['anima cosmica',
 'cosmica essenza',
 'cosmica essenza',
 'anima cosmica',
 'anima cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'cosmica essenza',
 'anima cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'cosmica essenza',
 'anima cosmica',
 'anima cosmica',
 'anima cosmica',
 'anima cosmica',
 'anima cosmica',
 'anima cosmica',
 'l essenza cosmica',
 'l essenza cosmica',
 'anima cosmica']

[]

In [36]:
import chromadb
chroma_client = chromadb.Client()

In [37]:
collection = chroma_client.create_collection(name="my_collection")

In [46]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id3", "id5"],
    metadatas=[{"fruit": ["pineapple"]}, {"fruit": "orange"}],
)

ValueError: Expected metadata value to be a str, int, float or bool, got ['pineapple'] which is a list in add.

In [34]:


# Example documents
documents = [
    "Document about regression and classification",
    "Document about CNN architecture",
    "Document about both regression and CNN"
]

# Method 1: Using lists in metadata (Recommended)
metadata_list = [
    {
        "techniques": ["regression", "classification"],
        "difficulty": "intermediate"
    },
    {
        "techniques": ["CNN"],
        "difficulty": "advanced"
    },
    {
        "techniques": ["regression", "CNN"],
        "difficulty": "advanced"
    }
]

# Add documents with metadata using lists
collection.add(
    documents=documents,
    ids=["doc1", "doc2", "doc3"],
    metadatas=metadata_list
)

# Query examples
# 1. Find documents about regression
results = collection.query(
    query_texts=["regression"],
    where={"techniques": {"$in": ["regression"]}}
)

# 2. Find advanced documents about CNN
results_advanced_cnn = collection.query(
    query_texts=["CNN architecture"],
    where={
        "$and": [
            {"techniques": {"$in": ["CNN"]}},
            {"difficulty": "advanced"}
        ]
    }
)